In [2]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<table align="left">

  <td>
    <a href="https://console.cloud.google.com/vertex-ai/notebooks/deploy-notebook?download_url=https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/master/notebooks/community/matching_engine/matching_engine_for_indexing.ipynb">
      Run in Google Cloud Notebooks
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/master/notebooks/community/matching_engine/matching_engine_for_indexing.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
</table>

## Overview

This example demonstrates how to use the GCP ANN Service. It is a high scale, low latency solution, to find similar vectors (or more specifically "embeddings") for a large corpus. Moreover, it is a fully managed offering, further reducing operational overhead. It is built upon [Approximate Nearest Neighbor (ANN) technology](https://ai.googleblog.com/2020/07/announcing-scann-efficient-vector.html) developed by Google Research.

### Dataset

The dataset used for this tutorial is the [GloVe dataset](https://nlp.stanford.edu/projects/glove/).

"GloVe is an unsupervised learning algorithm for obtaining vector representations for words. Training is performed on aggregated global word-word co-occurrence statistics from a corpus, and the resulting representations showcase interesting linear substructures of the word vector space."

### Objective

In this notebook, you will learn how to create Approximate Nearest Neighbor (ANN) Index, query against indexes, and validate the performance of the index. 

The steps performed include:

* Create ANN Index and Brute Force Index
* Create an IndexEndpoint with VPC Network
* Deploy ANN Index and Brute Force Index
* Perform online query
* Compute recall


### Costs 

This tutorial uses billable components of Google Cloud:

* Vertex AI
* Cloud Storage

Learn about [Vertex AI
pricing](https://cloud.google.com/vertex-ai/pricing) and [Cloud Storage
pricing](https://cloud.google.com/storage/pricing), and use the [Pricing
Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

## Before you begin

* **Prepare a VPC network**.  To reduce any network overhead that might lead to unnecessary increase in overhead latency, it is best to call the ANN endpoints from your VPC via a direct [VPC Peering](https://cloud.google.com/vertex-ai/docs/general/vpc-peering) connection. The following section describes how to setup a VPC Peering connection if you don't have one. This is a one-time initial setup task. You can also reuse existing VPC network and skip this section.
* **WARNING:** The match service gRPC API (to create online queries against your deployed index) has to be executed in a Google Cloud Notebook instance that is created with the following requirements:
  * **In the same region as where your ANN service is deployed** (for example, if you set `REGION = "us-central1"` as same as the tutorial, the notebook instance has to be in `us-central1`).
  * **Make sure you select the VPC network you created for ANN service** (instead of using the "default" one). That is, you will have to create the VPC network below and then create a new notebook instance that uses that VPC.  
  * If you run it in the colab or a Google Cloud Notebook instance in a different VPC network or region, the gRPC API will fail to peer the network (InactiveRPCError).

In [1]:
PROJECT_ID = "python-docs-samples-tests"  # @param {type:"string"}

NETWORK_NAME = "ucaip-haystack-vpc-network"  # @param {type:"string"}

PEERING_RANGE_NAME = "ucaip-haystack-range"

In [9]:
# Create a VPC network
! gcloud compute networks create {NETWORK_NAME} --bgp-routing-mode=regional --subnet-mode=auto --project={PROJECT_ID}

# Add necessary firewall rules
! gcloud compute firewall-rules create {NETWORK_NAME}-allow-icmp --network {NETWORK_NAME} --priority 65534 --project {PROJECT_ID} --allow icmp

! gcloud compute firewall-rules create {NETWORK_NAME}-allow-internal --network {NETWORK_NAME} --priority 65534 --project {PROJECT_ID} --allow all --source-ranges 10.128.0.0/9

! gcloud compute firewall-rules create {NETWORK_NAME}-allow-rdp --network {NETWORK_NAME} --priority 65534 --project {PROJECT_ID} --allow tcp:3389

! gcloud compute firewall-rules create {NETWORK_NAME}-allow-ssh --network {NETWORK_NAME} --priority 65534 --project {PROJECT_ID} --allow tcp:22

# Reserve IP range
! gcloud compute addresses create {PEERING_RANGE_NAME} --global --prefix-length=16 --network={NETWORK_NAME} --purpose=VPC_PEERING --project={PROJECT_ID} --description="peering range for uCAIP Haystack."

# Set up peering with service networking
! gcloud services vpc-peerings connect --service=servicenetworking.googleapis.com --network={NETWORK_NAME} --ranges={PEERING_RANGE_NAME} --project={PROJECT_ID}

ERROR: (gcloud.compute.networks.create) Could not fetch resource:
 - The resource 'projects/python-docs-samples-tests/regions/us-central1/subnetworks/ucaip-haystack-vpc-network' already exists

ERROR: (gcloud.compute.firewall-rules.create) Could not fetch resource:
 - The resource 'projects/python-docs-samples-tests/global/firewalls/ucaip-haystack-vpc-network-allow-icmp' already exists

ERROR: (gcloud.compute.firewall-rules.create) Could not fetch resource:
 - The resource 'projects/python-docs-samples-tests/global/firewalls/ucaip-haystack-vpc-network-allow-internal' already exists

ERROR: (gcloud.compute.firewall-rules.create) Could not fetch resource:
 - The resource 'projects/python-docs-samples-tests/global/firewalls/ucaip-haystack-vpc-network-allow-rdp' already exists

ERROR: (gcloud.compute.firewall-rules.create) Could not fetch resource:
 - The resource 'projects/python-docs-samples-tests/global/firewalls/ucaip-haystack-vpc-network-allow-ssh' already exists

ERROR: (gcloud.compu

* Authentication: `$ gcloud auth login` rerun this in Google Cloud Notebook terminal when you are logged out and need the credential again.

### Installation

Download and install the latest (preview) version of the Vertex SDK for Python.

In [19]:
! pip install -U git+https://github.com/ivanmkc/python-aiplatform.git@imkc--matching-engine

  Cloning https://github.com/ivanmkc/python-aiplatform.git (to revision imkc--matching-engine) to /tmp/pip-req-build-c3a7oc5o
  Running command git clone -q https://github.com/ivanmkc/python-aiplatform.git /tmp/pip-req-build-c3a7oc5o
  Running command git checkout -b imkc--matching-engine --track origin/imkc--matching-engine
  Switched to a new branch 'imkc--matching-engine'
  Branch 'imkc--matching-engine' set up to track remote branch 'imkc--matching-engine' from 'origin'.
     |████████████████████████████████| 46 kB 2.1 MB/s 
     |████████████████████████████████| 106 kB 9.2 MB/s 
     |████████████████████████████████| 106 kB 64.8 MB/s 
     |████████████████████████████████| 106 kB 60.0 MB/s 
     |████████████████████████████████| 106 kB 42.5 MB/s 
     |████████████████████████████████| 105 kB 80.1 MB/s 
     |████████████████████████████████| 105 kB 63.2 MB/s 
     |████████████████████████████████| 105 kB 62.0 MB/s 
     |████████████████████████████████| 105 kB 65.2 MB/s 
 

Install the `h5py` to prepare sample dataset, and the `grpcio-tools` for querying against the index. 

In [1]:
! pip install -U grpcio-tools
! pip install -U h5py

     |████████████████████████████████| 2.4 MB 5.2 MB/s 
     |████████████████████████████████| 4.3 MB 42.5 MB/s 
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.43.0
    Uninstalling grpcio-1.43.0:
      Successfully uninstalled grpcio-1.43.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.0 requires tf-estimator-nightly==2.8.0.dev2021122109, which is not installed.
     |████████████████████████████████| 4.1 MB 4.9 MB/s 
  Attempting uninstall: h5py
    Found existing installation: h5py 3.1.0
    Uninstalling h5py-3.1.0:
      Successfully uninstalled h5py-3.1.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.0 requires tf-estimator-nightly==2.8.0.dev2021122109, which is not inst

### Restart the kernel

After you install the additional packages, you need to restart the notebook kernel so it can find the packages.

In [2]:
# Automatically restart kernel after installs
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

### Set up your Google Cloud project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a Google Cloud project](https://console.cloud.google.com/cloud-resource-manager).

1. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).

1. [Enable the Vertex AI API and Compute Engine API, and Service Networking API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,compute_component,servicenetworking.googleapis.com).

1. Enter your project ID in the cell below. Then run the cell to make sure the
Cloud SDK uses the right project for all the commands in this notebook.

**Note**: Jupyter runs lines prefixed with `!` as shell commands, and it interpolates Python variables prefixed with `$` into these commands.

#### Set your project ID

**If you don't know your project ID**, you may be able to get your project ID using `gcloud`.

In [ ]:
import os

PROJECT_ID = ""

# Get your Google Cloud project ID from gcloud
if not os.getenv("IS_TESTING"):
    shell_output=!gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID: ", PROJECT_ID)

Otherwise, set your project ID here.

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None:
    PROJECT_ID = "python-docs-samples-tests"  # @param {type:"string"}

#### Timestamp

If you are in a live tutorial session, you might be using a shared test account or project. To avoid name collisions between users on resources created, you create a timestamp for each instance session, and append it onto the name of resources you create in this tutorial.

In [3]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### Authenticate your Google Cloud account

**If you are using Google Cloud Notebooks**, your environment is already
authenticated. Skip this step.

**If you are using Colab**, run the cell below and follow the instructions
when prompted to authenticate your account via oAuth.

**Otherwise**, follow these steps:

1. In the Cloud Console, go to the [**Create service account key**
   page](https://console.cloud.google.com/apis/credentials/serviceaccountkey).

2. Click **Create service account**.

3. In the **Service account name** field, enter a name, and
   click **Create**.

4. In the **Grant this service account access to project** section, click the **Role** drop-down list. Type "Vertex AI"
into the filter box, and select
   **Vertex AI Administrator**. Type "Storage Object Admin" into the filter box, and select **Storage Object Admin**.

5. Click *Create*. A JSON file that contains your key downloads to your
local environment.

6. Enter the path to your service account key as the
`GOOGLE_APPLICATION_CREDENTIALS` variable in the cell below and run the cell.

In [4]:
import os
import sys

# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

# The Google Cloud Notebook product has specific requirements
IS_GOOGLE_CLOUD_NOTEBOOK = os.path.exists("/opt/deeplearning/metadata/env_version")

# If on Google Cloud Notebooks, then don't execute this code
if not IS_GOOGLE_CLOUD_NOTEBOOK:
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, log in using gcloud
    elif not os.getenv("IS_TESTING"):
        ! gcloud auth login

### Create a Cloud Storage bucket

**The following steps are required, regardless of your notebook environment.**

Set the name of your Cloud Storage bucket below. It must be unique across all
Cloud Storage buckets.

You may also change the `REGION` variable, which is used for operations
throughout the rest of this notebook. Make sure to [choose a region where Vertex AI services are
available](https://cloud.google.com/vertex-ai/docs/general/locations#available_regions). You may
not use a Multi-Regional Storage bucket for training with Vertex AI.

In [5]:
BUCKET_URI = "gs://[your-bucket-name]"  # @param {type:"string"}
REGION = "[your-region]"  # @param {type:"string"}

In [6]:
if BUCKET_URI == "" or BUCKET_URI is None or BUCKET_URI == "gs://[your-bucket-name]":
    BUCKET_URI = "gs://" + PROJECT_ID + "aip-" + TIMESTAMP

if REGION == "[your-region]":
    REGION = "us-central1"

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [7]:
! gsutil mb -l $REGION -p $PROJECT_ID $BUCKET_URI

Creating gs://python-docs-samples-testsaip-20220224190650/...


Finally, validate access to your Cloud Storage bucket by examining its contents:

In [8]:
! gsutil ls -al $BUCKET_URI

### Import libraries and define constants

Import the Vertex AI (unified) client library into your Python environment. 


In [9]:
import h5py

In [10]:
PROJECT_NUMBER = !gcloud projects list --filter="PROJECT_ID:'{PROJECT_ID}'" --format='value(PROJECT_NUMBER)'
PROJECT_NUMBER = PROJECT_NUMBER[0]

PARENT = "projects/{}/locations/{}".format(PROJECT_ID, REGION)

print("PROJECT_ID: {}".format(PROJECT_ID))
print("REGION: {}".format(REGION))

!gcloud config set project {PROJECT_ID}
!gcloud config set ai_platform/region {REGION}

PROJECT_ID: python-docs-samples-tests
REGION: us-central1
Updated property [core/project].
Updated property [ai_platform/region].


## Prepare the Data

The GloVe dataset consists of a set of pre-trained embeddings. The embeddings are split into a "train" split, and a "test" split.
We will create a vector search index from the "train" split, and use the embedding vectors in the "test" split as query vectors to test the vector search index.

NOTE: While the data split uses the term "train", these are pre-trained embeddings and thus are ready to be indexed for search. The terms "train" and "test" split are used just to be consistent with usual machine learning terminology.

Download the GloVe dataset.


In [11]:
! gsutil cp gs://cloud-samples-data/vertex-ai/matching_engine/glove-100-angular.hdf5 .

Copying gs://cloud-samples-data/vertex-ai/matching_engine/glove-100-angular.hdf5...
/ [1 files][462.9 MiB/462.9 MiB]                                                
Operation completed over 1 objects/462.9 MiB.                                    


Read the data into memory.


In [12]:
# The number of nearest neighbors to be retrieved from database for each query.
NUM_NEIGHBOURS = 10

h5 = h5py.File("glove-100-angular.hdf5", "r")
train = h5["train"]
test = h5["test"]

In [13]:
train[0]

array([-0.11333  ,  0.48402  ,  0.090771 , -0.22439  ,  0.034206 ,
       -0.55831  ,  0.041849 , -0.53573  ,  0.18809  , -0.58722  ,
        0.015313 , -0.014555 ,  0.80842  , -0.038519 ,  0.75348  ,
        0.70502  , -0.17863  ,  0.3222   ,  0.67575  ,  0.67198  ,
        0.26044  ,  0.4187   , -0.34122  ,  0.2286   , -0.53529  ,
        1.2582   , -0.091543 ,  0.19716  , -0.037454 , -0.3336   ,
        0.31399  ,  0.36488  ,  0.71263  ,  0.1307   , -0.24654  ,
       -0.52445  , -0.036091 ,  0.55068  ,  0.10017  ,  0.48095  ,
        0.71104  , -0.053462 ,  0.22325  ,  0.30917  , -0.39926  ,
        0.036634 , -0.35431  , -0.42795  ,  0.46444  ,  0.25586  ,
        0.68257  , -0.20821  ,  0.38433  ,  0.055773 , -0.2539   ,
       -0.20804  ,  0.52522  , -0.11399  , -0.3253   , -0.44104  ,
        0.17528  ,  0.62255  ,  0.50237  , -0.7607   , -0.071786 ,
        0.0080131, -0.13286  ,  0.50097  ,  0.18824  , -0.54722  ,
       -0.42664  ,  0.4292   ,  0.14877  , -0.0072514, -0.1648

Save the train split in JSONL format.


In [14]:
with open("glove100.json", "w") as f:
    for i in range(len(train)):
        f.write('{"id":"' + str(i) + '",')
        f.write('"embedding":[' + ",".join(str(x) for x in train[i]) + "]}")
        f.write("\n")

Upload the training data to GCS.

In [15]:
EMBEDDINGS_INITIAL_URI = f"{BUCKET_URI}/matching_engine/initial/"
! gsutil cp glove100.json {EMBEDDINGS_INITIAL_URI}

Copying file://glove100.json [Content-Type=application/json]...
==> NOTE: You are uploading one or more large file(s), which would run
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

/ [1 files][987.2 MiB/987.2 MiB]                                                
Operation completed over 1 objects/987.2 MiB.                                    


## Create Indexes


### Create ANN Index (for Production Usage)

In [16]:
DIMENSIONS = 100
DISPLAY_NAME = "glove_100_1"
DISPLAY_NAME_BRUTE_FORCE = DISPLAY_NAME + "_brute_force"

Create the ANN index configuration:

Please read the documentation to understand the various configuration parameters that can be used to tune the index


In [17]:
import os
import sys

from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=BUCKET_URI)

In [ ]:
tree_ah_index = aiplatform.MatchingEngineIndex.create_tree_ah_index(
    display_name=DISPLAY_NAME,
    contents_delta_uri=EMBEDDINGS_INITIAL_URI,
    dimensions=DIMENSIONS,
    approximate_neighbors_count=150,
    distance_measure_type="DOT_PRODUCT_DISTANCE",
    leaf_node_embedding_count=500,
    leaf_nodes_to_search_percent=7,
    description="Glove 100 ANN index",
    labels={ "label_name": "label_value" },
)

INFO:google.cloud.aiplatform.matching_engine.matching_engine_index:Creating MatchingEngineIndex
INFO:google.cloud.aiplatform.matching_engine.matching_engine_index:Create MatchingEngineIndex backing LRO: projects/1012616486416/locations/us-central1/indexes/8500561888847855616/operations/5645970836872495104


In [ ]:
INDEX_RESOURCE_NAME = tree_ah_index.resource_name
INDEX_RESOURCE_NAME

### Create Brute Force Index (for Ground Truth)

The brute force index uses a naive brute force method to find the nearest neighbors. This method is not fast or efficient. Hence brute force indices are not recommended for production usage. They are to be used to find the "ground truth" set of neighbors, so that the "ground truth" set can be used to measure recall of the indices being tuned for production usage. To ensure an apples to apples comparison, the `distanceMeasureType` and `featureNormType`, `dimensions` of the brute force index should match those of the production indices being tuned.

Create the brute force index configuration:

In [ ]:
brute_force_index = aiplatform.MatchingEngineIndex.create_brute_force_index(
    display_name=DISPLAY_NAME,
    contents_delta_uri=EMBEDDINGS_INITIAL_URI,
    dimensions=DIMENSIONS,
    approximate_neighbors_count=150,
    distance_measure_type="DOT_PRODUCT_DISTANCE",
    description="Glove 100 index (brute force)",
    labels={ "label_name": "label_value" },
)

In [ ]:
INDEX_BRUTE_FORCE_RESOURCE_NAME = brute_force_index.resource_name
INDEX_BRUTE_FORCE_RESOURCE_NAME

## Update Indexes

Create incremental data file.


In [ ]:
with open("glove100_incremental.json", "w") as f:
    f.write(
        '{"id":"0","embedding":[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]}\n'
    )

Copy the incremental data file to a new subdirectory.


In [ ]:
EMBEDDINGS_UPDATE_URI = f"{BUCKET_URI}/matching-engine/incremental/"

In [ ]:
! gsutil cp glove100_incremental.json {EMBEDDINGS_UPDATE_URI}

Create update index request


In [ ]:
tree_ah_index = tree_ah_index.update_embeddings(
            contents_delta_uri=EMBEDDINGS_UPDATE_URI,
        )

In [ ]:
INDEX_RESOURCE_NAME = tree_ah_index.resource_name
INDEX_RESOURCE_NAME

## Create an IndexEndpoint with VPC Network

In [ ]:
VPC_NETWORK_NAME = "projects/{}/global/networks/{}".format(PROJECT_NUMBER, NETWORK_NAME)
VPC_NETWORK_NAME

In [ ]:
my_index_endpoint = aiplatform.MatchingEngineIndexEndpoint.create(
    display_name="index_endpoint_for_demo",
    description="index endpoint description",
    network=VPC_NETWORK_NAME,
)

In [ ]:
my_index_endpoint = aiplatform.MatchingEngineIndexEndpoint(index_endpoint_name="projects/1012616486416/locations/us-central1/indexEndpoints/1370185002255384576")

In [ ]:
INDEX_ENDPOINT_NAME = my_index_endpoint.resource_name
INDEX_ENDPOINT_NAME

## Deploy Indexes

### Deploy ANN Index

In [ ]:
DEPLOYED_INDEX_ID = "tree_ah_glove_deployed"

In [ ]:
my_index_endpoint = my_index_endpoint.deploy_index(index=tree_ah_index, deployed_index_id=DEPLOYED_INDEX_ID)

my_index_endpoint.deployed_indexes

### Deploy Brute Force Index

In [ ]:
DEPLOYED_BRUTE_FORCE_INDEX_ID = "glove_brute_force_deployed"

In [ ]:
my_index_endpoint = my_index_endpoint.deploy_index(index=brute_force_index, deployed_index_id=DEPLOYED_BRUTE_FORCE_INDEX_ID)

my_index_endpoint.deployed_indexes

## Create Online Queries

After you built your indexes, you may query against the deployed index through the online querying gRPC API (Match service) within the virtual machine instances from the same region (for example 'us-central1' in this tutorial).  

Test your query:

In [ ]:
# Test query
query = [
    -0.11333,
    0.48402,
    0.090771,
    -0.22439,
    0.034206,
    -0.55831,
    0.041849,
    -0.53573,
    0.18809,
    -0.58722,
    0.015313,
    -0.014555,
    0.80842,
    -0.038519,
    0.75348,
    0.70502,
    -0.17863,
    0.3222,
    0.67575,
    0.67198,
    0.26044,
    0.4187,
    -0.34122,
    0.2286,
    -0.53529,
    1.2582,
    -0.091543,
    0.19716,
    -0.037454,
    -0.3336,
    0.31399,
    0.36488,
    0.71263,
    0.1307,
    -0.24654,
    -0.52445,
    -0.036091,
    0.55068,
    0.10017,
    0.48095,
    0.71104,
    -0.053462,
    0.22325,
    0.30917,
    -0.39926,
    0.036634,
    -0.35431,
    -0.42795,
    0.46444,
    0.25586,
    0.68257,
    -0.20821,
    0.38433,
    0.055773,
    -0.2539,
    -0.20804,
    0.52522,
    -0.11399,
    -0.3253,
    -0.44104,
    0.17528,
    0.62255,
    0.50237,
    -0.7607,
    -0.071786,
    0.0080131,
    -0.13286,
    0.50097,
    0.18824,
    -0.54722,
    -0.42664,
    0.4292,
    0.14877,
    -0.0072514,
    -0.16484,
    -0.059798,
    0.9895,
    -0.61738,
    0.054169,
    0.48424,
    -0.35084,
    -0.27053,
    0.37829,
    0.11503,
    -0.39613,
    0.24266,
    0.39147,
    -0.075256,
    0.65093,
    -0.20822,
    -0.17456,
    0.53571,
    -0.16537,
    0.13582,
    -0.56016,
    0.016964,
    0.1277,
    0.94071,
    -0.22608,
    -0.021106,
]

response = my_index_endpoint.match(deployed_index_id=DEPLOYED_INDEX_ID, queries=[query], num_neighbors=NUM_NEIGHBOURS)

response

### Batch Query

You can run multiple queries in a single match call:

In [ ]:
# Test query
queries = [
    [
        -0.11333,
        0.48402,
        0.090771,
        -0.22439,
        0.034206,
        -0.55831,
        0.041849,
        -0.53573,
        0.18809,
        -0.58722,
        0.015313,
        -0.014555,
        0.80842,
        -0.038519,
        0.75348,
        0.70502,
        -0.17863,
        0.3222,
        0.67575,
        0.67198,
        0.26044,
        0.4187,
        -0.34122,
        0.2286,
        -0.53529,
        1.2582,
        -0.091543,
        0.19716,
        -0.037454,
        -0.3336,
        0.31399,
        0.36488,
        0.71263,
        0.1307,
        -0.24654,
        -0.52445,
        -0.036091,
        0.55068,
        0.10017,
        0.48095,
        0.71104,
        -0.053462,
        0.22325,
        0.30917,
        -0.39926,
        0.036634,
        -0.35431,
        -0.42795,
        0.46444,
        0.25586,
        0.68257,
        -0.20821,
        0.38433,
        0.055773,
        -0.2539,
        -0.20804,
        0.52522,
        -0.11399,
        -0.3253,
        -0.44104,
        0.17528,
        0.62255,
        0.50237,
        -0.7607,
        -0.071786,
        0.0080131,
        -0.13286,
        0.50097,
        0.18824,
        -0.54722,
        -0.42664,
        0.4292,
        0.14877,
        -0.0072514,
        -0.16484,
        -0.059798,
        0.9895,
        -0.61738,
        0.054169,
        0.48424,
        -0.35084,
        -0.27053,
        0.37829,
        0.11503,
        -0.39613,
        0.24266,
        0.39147,
        -0.075256,
        0.65093,
        -0.20822,
        -0.17456,
        0.53571,
        -0.16537,
        0.13582,
        -0.56016,
        0.016964,
        0.1277,
        0.94071,
        -0.22608,
        -0.021106,
    ],
    [
        -0.99544,
        -2.3651,
        -0.24332,
        -1.0321,
        0.42052,
        -1.1817,
        -0.16451,
        -1.683,
        0.49673,
        -0.27258,
        -0.025397,
        0.34188,
        1.5523,
        1.3532,
        0.33297,
        -0.0056677,
        -0.76525,
        0.49587,
        1.2211,
        0.83394,
        -0.20031,
        -0.59657,
        0.38485,
        -0.23487,
        -1.0725,
        0.95856,
        0.16161,
        -1.2496,
        1.6751,
        0.73899,
        0.051347,
        -0.42702,
        0.16257,
        -0.16772,
        0.40146,
        0.29837,
        0.96204,
        -0.36232,
        -0.47848,
        0.78278,
        0.14834,
        1.3407,
        0.47834,
        -0.39083,
        -1.037,
        -0.24643,
        -0.75841,
        0.7669,
        -0.37363,
        0.52741,
        0.018563,
        -0.51301,
        0.97674,
        0.55232,
        1.1584,
        0.73715,
        1.3055,
        -0.44743,
        -0.15961,
        0.85006,
        -0.34092,
        -0.67667,
        0.2317,
        1.5582,
        1.2308,
        -0.62213,
        -0.032801,
        0.1206,
        -0.25899,
        -0.02756,
        -0.52814,
        -0.93523,
        0.58434,
        -0.24799,
        0.37692,
        0.86527,
        0.069626,
        1.3096,
        0.29975,
        -1.3651,
        -0.32048,
        -0.13741,
        0.33329,
        -1.9113,
        -0.60222,
        -0.23921,
        0.12664,
        -0.47961,
        -0.89531,
        0.62054,
        0.40869,
        -0.08503,
        0.6413,
        -0.84044,
        -0.74325,
        -0.19426,
        0.098722,
        0.32648,
        -0.67621,
        -0.62692,
    ],
]

### Compute Recall

Use deployed brute force Index as the ground truth to calculate the recall of ANN Index:

In [ ]:
# Retrieve nearest neighbors for both the tree-AH index and the brute-force index
tree_ah_response_test = my_index_endpoint.match(deployed_index_id=DEPLOYED_INDEX_ID, queries=list(test), num_neighbors=NUM_NEIGHBOURS)
brute_force_response_test = my_index_endpoint.match(deployed_index_id=DEPLOYED_BRUTE_FORCE_INDEX_ID, queries=list(test), num_neighbors=NUM_NEIGHBOURS)

In [ ]:
# Calculate recall by determining how many neighbors were correctly retrieved as compared to the brute-force option.
correct_neighbors = 0
for tree_ah_neighbors, brute_force_neighbors in zip(tree_ah_response_test, brute_force_response_test):
    tree_ah_neighbor_ids = [neighbor.id for neighbor in tree_ah_neighbors]
    brute_force_neighbor_ids = [neighbor.id for neighbor in brute_force_neighbors]
    
    correct_neighbors += len(set(tree_ah_neighbor_ids).intersection(brute_force_neighbor_ids))

recall = correct_neighbors / (len(test) * NUM_NEIGHBOURS)

print("Recall: {}".format(recall))

## Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.
You can also manually delete resources that you created by running the following code.

In [ ]:
# Force undeployment of indexes and delete endpoint
my_index_endpoint.delete(force=True)

In [ ]:
# Delete indexes
tree_ah_index.delete(force=True)
brute_force_index.delete(force=True)